In [2]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets

In [3]:
#Hyperparameter
batch_size = 128
learning_rate = 1e-2
num_epoches = 20

In [4]:
train_dataset = datasets.MNIST(
    root='./data', train=True, transform=transforms.ToTensor(), download=True)

test_dataset = datasets.MNIST(
    root='./data', train=False, transform=transforms.ToTensor())

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [14]:
class CNN(nn.Module):
    def __init__(self, in_dim, n_class):
        super(CNN, self).__init__()
        self.conv=nn.Sequential(
            nn.Conv2d(in_dim,6,3,stride=1,padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(2,2),
            nn.Conv2d(6,16,5,stride=1,padding=0),
            nn.ReLU(True),
            nn.MaxPool2d((2,2)))

        self.fc = nn.Sequential(
            nn.Linear(400, 120), nn.Linear(120, 84), nn.Linear(84, n_class))
    def forward(self,x):
            out = self.conv(x)
            out = out.view(out.size(0),-1)
            out = self.fc(out)
            return out


In [16]:
model = CNN(1, 10)
model = model.cuda()

In [18]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [21]:
for epoch in range(num_epoches):
    print('epoch{}'.format(epoch+1))
    print('*'*10)
    running_loss=0.0
    running_acc=0.0
    for i,data in enumerate(train_loader,1):
        img,label=data
        img = img.cuda()
        label = label.cuda()
        img = Variable(img)
        label = Variable(label)
        #forward
        out=model(img)
        loss=criterion(out,label)
        running_loss+=loss.data[0]*label.size(0)
        _,pred=torch.max(out,1)
        num_correct=(pred==label).sum()
        accuracy=(pred==label).float().mean()
        running_acc +=num_correct.data[0]
        #backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #
        if i % 300 == 0:
            print('[{}/{}] Loss: {:.6f}, Acc: {:.6f}'.format(
                epoch + 1, num_epoches, running_loss / (batch_size * i),
                running_acc / (batch_size * i)))
    print('Finish {} epoch, Loss: {:.6f}, Acc: {:.6f}'.format(
        epoch + 1, running_loss / (len(train_dataset)), running_acc / (len(
            train_dataset))))
    model.eval()
    eval_loss = 0
    eval_acc = 0
    for data in test_loader:
        img, label = data

        img = Variable(img, volatile=True).cuda()
        label = Variable(label, volatile=True).cuda()

        out = model(img)
        loss = criterion(out, label)
        eval_loss += loss.data[0] * label.size(0)
        _, pred = torch.max(out, 1)
        num_correct = (pred == label).sum()
        eval_acc += num_correct.data[0]
    print('Test Loss: {:.6f}, Acc: {:.6f}'.format(eval_loss / (len(
        test_dataset)), eval_acc / (len(test_dataset))))
    print()

# savemodel
torch.save(model.state_dict(), './cnn.pth')

epoch1
**********
[1/20] Loss: 2.295023, Acc: 0.155573
Finish 1 epoch, Loss: 2.279683, Acc: 0.258617
Test Loss: 2.204051, Acc: 0.550800

epoch2
**********
[2/20] Loss: 1.488435, Acc: 0.688385
Finish 2 epoch, Loss: 1.141745, Acc: 0.745367
Test Loss: 0.437699, Acc: 0.871400

epoch3
**********
[3/20] Loss: 0.394851, Acc: 0.880703
Finish 3 epoch, Loss: 0.369296, Acc: 0.888967
Test Loss: 0.293190, Acc: 0.910600

epoch4
**********
[4/20] Loss: 0.284822, Acc: 0.915260
Finish 4 epoch, Loss: 0.275181, Acc: 0.917767
Test Loss: 0.226747, Acc: 0.930600

epoch5
**********
[5/20] Loss: 0.224022, Acc: 0.932057
Finish 5 epoch, Loss: 0.217302, Acc: 0.934833
Test Loss: 0.178985, Acc: 0.946000

epoch6
**********
[6/20] Loss: 0.186569, Acc: 0.943490
Finish 6 epoch, Loss: 0.178512, Acc: 0.946283
Test Loss: 0.151485, Acc: 0.954500

epoch7
**********
[7/20] Loss: 0.155275, Acc: 0.953646
Finish 7 epoch, Loss: 0.152282, Acc: 0.954000
Test Loss: 0.130105, Acc: 0.959400

epoch8
**********
[8/20] Loss: 0.136499, 